In [18]:
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.6.0.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0.html
  Using cached https://pytorch-geometric.com/whl/torch-1.6.0/torch_scatter-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-scatter 2.0.5
    Uninstalling torch-scatter-2.0.5:
      Successfully uninstalled torch-scatter-2.0.5


Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0.html
  Using cached https://pytorch-geometric.com/whl/torch-1.6.0/torch_sparse-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-sparse 0.6.7
    Uninstalling torch-sparse-0.6.7:
      Successfully uninstalled torch-sparse-0.6.7


Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0.html
  Using cached https://pytorch-geometric.com/whl/torch-1.6.0/torch_cluster-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-cluster 1.5.7
    Uninstalling torch-cluster-1.5.7:
      Successfully uninstalled torch-cluster-1.5.7


Looking in links: https://pytorch-geometric.com/whl/torch-1.6.0.html
  Using cached https://pytorch-geometric.com/whl/torch-1.6.0/torch_spline_conv-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-spline-conv 1.2.0
    Uninstalling torch-spline-conv-1.2.0:
      Successfully uninstalled torch-spline-conv-1.2.0


In [19]:
import torch
import torch.nn as n
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as f
import random
import torch.optim as op

In [20]:
import torch_geometric.nn as tg
import torch_geometric.data as Data

In [21]:
from torch_geometric.utils import degree,add_self_loops

In [105]:
class GCN(tg.MessagePassing):
  def __init__(self,inchannels,outchannels):
    super().__init__(aggr="mean")
    self.fc=n.Linear(inchannels,outchannels)

  def forward(self,x,edgeindex):
    # x has shape N x nchannels
    # edgeindex shape 2 x E 
    # adding self loops to adjacency matrix
    edgeindex,_=add_self_loops(edgeindex,num_nodes=x.shape[0])
    x=self.fc(x)  #Feed forward neural network
    row,col=edgeindex
    degreecalc=degree(col,num_nodes=x.shape[0],dtype=x.dtype)
    print("######")
    print(col)
    print(row)
    print("************")
    print(degreecalc)
    print(degreecalc.shape)
    deginverse=degreecalc.pow(-0.5)
    print("####")
    print(deginverse)
    norm=deginverse[row]*deginverse[col]
    print(norm.shape)      # we added self loops
    print("$$$$")
    print(deginverse[row])
    return self.propagate(edgeindex,x=x,norm=norm)

  def message(self, x_j, norm):
    # x_j has shape [E, out_channels]

    # Step 4: Normalize node features.
    return norm.view(-1, 1) * x_j



In [94]:
from torch_geometric.datasets import KarateClub

In [67]:
dataset = KarateClub()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

Dataset: KarateClub():
Number of graphs: 1
Number of features: 34
Number of classes: 2


In [52]:
data=dataset[0]

In [53]:
print("total no of nodes are ",data.num_nodes)
print("feature dimension ",data.num_features)
print("edge dimension ",data.num_edge_features)
print("number of edges ",data.num_edges)

total no of nodes are  34
feature dimension  34
edge dimension  0
number of edges  156


In [54]:
xdata=data.x

In [55]:
ydata=data.y

In [56]:
xdata.shape

torch.Size([34, 34])

In [57]:
ydata

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [58]:
reldata=data.edge_index

In [59]:
reldata.shape

torch.Size([2, 156])

In [60]:
relfeatures=data.edge_attr

In [61]:
indim=34

In [106]:
model=GCN(34,16)

In [107]:
model

GCN(
  (fc): Linear(in_features=34, out_features=16, bias=True)
)

In [108]:
#col has nodes which are increasing so as the number decrease implies we are starting new

p=model.forward(xdata,reldata)

######
tensor([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 17, 19, 21, 31,  0,  2,
         3,  7, 13, 17, 19, 21, 30,  0,  1,  3,  7,  8,  9, 13, 27, 28, 32,  0,
         1,  2,  7, 12, 13,  0,  6, 10,  0,  6, 10, 16,  0,  4,  5, 16,  0,  1,
         2,  3,  0,  2, 30, 32, 33,  2, 33,  0,  4,  5,  0,  0,  3,  0,  1,  2,
         3, 33, 32, 33, 32, 33,  5,  6,  0,  1, 32, 33,  0,  1, 33, 32, 33,  0,
         1, 32, 33, 25, 27, 29, 32, 33, 25, 27, 31, 23, 24, 31, 29, 33,  2, 23,
        24, 33,  2, 31, 33, 23, 26, 32, 33,  1,  8, 32, 33,  0, 24, 25, 28, 32,
        33,  2,  8, 14, 15, 18, 20, 22, 23, 29, 30, 31, 33,  8,  9, 13, 14, 15,
        18, 19, 20, 22, 23, 26, 27, 28, 29, 30, 31, 32,  0,  1,  2,  3,  4,  5,
         6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
        24, 25, 26, 27, 28, 29, 30, 31, 32, 33])
tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2, 